In [3]:
# librerias
import pandas as pd

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mkechinov/ecommerce-behavior-data-from-multi-category-store")

print("Path to dataset files:", path)

c:\Users\Joabh\anaconda3\envs\proyectoDS\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Resuming download from 3889168384 bytes (717552523 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/mkechinov/ecommerce-behavior-data-from-multi-category-store?dataset_version_number=8 (3889168384/4606720907) bytes left.


100%|██████████| 4.29G/4.29G [05:04<00:00, 2.36MB/s]

Extracting files...


Path to dataset files: C:\Users\Joabh\.cache\kagglehub\datasets\mkechinov\ecommerce-behavior-data-from-multi-category-store\versions\8


In [7]:

# Path del dataset
dataset_path = "C:/Users/Joabh/.cache/kagglehub/datasets/mkechinov/ecommerce-behavior-data-from-multi-category-store/versions/8/2019-Nov.csv"

# Tamaño objetivo de la muestra
target_sample_size = 25000

# Lectura en chunks, muestreo aleatorio y acumulación
sampled_data = []  # Lista para guardar las muestras
chunk_size = 10**6  # Tamaño del chunk

# Iteramos sobre cada chunk del CSV
for chunk in pd.read_csv(dataset_path, chunksize=chunk_size, parse_dates=['event_time']):
    # Convertir event_time a datetime y extraer la hora
    chunk['hour'] = chunk['event_time'].dt.hour
    
    # Muestreo estratificado por hora (cada hora estará representada proporcionalmente)
    chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.01, random_state=1)).reset_index(drop=True)
    sampled_data.append(chunk_sample)
    
    # Si ya hemos alcanzado o superado el tamaño de la muestra objetivo, rompemos el ciclo
    if sum(len(sample) for sample in sampled_data) >= target_sample_size:
        break

# Concatenamos las muestras en un DataFrame
final_sample = pd.concat(sampled_data)

# Si tenemos más de 25k filas, tomamos una muestra aleatoria final
if len(final_sample) > target_sample_size:
    final_sample = final_sample.sample(n=target_sample_size, random_state=1)

# Guardar la muestra final a un archivo CSV
output_path = "C:/Users/Joabh/Documents/GitHub/ProyectoFInal_DS/sampled_data_25k.csv"
final_sample.to_csv(output_path, index=False)

print(f"Se ha generado una muestra aleatoria de {len(final_sample)} filas y se ha guardado en {output_path}.")




C:\Users\Joabh\AppData\Local\Temp\ipykernel_20772\2722909316.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.01, random_state=1)).reset_index(drop=True)
C:\Users\Joabh\AppData\Local\Temp\ipykernel_20772\2722909316.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  chunk_sample = chunk.groupby('hour').apply(lambda x: x.sample(frac=0.

Se ha generado una muestra aleatoria de 25000 filas y se ha guardado en C:/Users/Joabh/Documents/GitHub/ProyectoFInal_DS/sampled_data_25k.csv.
